### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Cargamos las Obras Completas de JL Borges

In [31]:
file = '/content/boc.csv'
df = pd.read_csv(file,sep='/n', header=None)
# Dataset is now stored in a Pandas Dataframe

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [32]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 36332


### 1 - Preprocesamiento

In [33]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [34]:
# Demos un vistazo
sentence_tokens[:2]

[['quiero',
  'dejar',
  'escrita',
  'una',
  'confesión',
  'que',
  'a',
  'un',
  'tiempo',
  'será'],
 ['íntima',
  'y',
  'general',
  'ya',
  'que',
  'las',
  'cosas',
  'que',
  'le',
  'ocurren',
  'a',
  'un']]

### 2 - Crear los vectores (word2vec)

In [35]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [36]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [37]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [38]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 36332


In [39]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 6861


### 3 - Entrenar el modelo generador

In [40]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 2179417.25
Loss after epoch 1: 1332969.0
Loss after epoch 2: 1284020.25
Loss after epoch 3: 1240830.0
Loss after epoch 4: 1229027.0
Loss after epoch 5: 1210015.5
Loss after epoch 6: 1166922.0
Loss after epoch 7: 1153147.0
Loss after epoch 8: 1140818.0
Loss after epoch 9: 1130526.0
Loss after epoch 10: 1115833.0
Loss after epoch 11: 1106948.0
Loss after epoch 12: 1096154.0
Loss after epoch 13: 1072073.0
Loss after epoch 14: 1051122.0
Loss after epoch 15: 1046726.0
Loss after epoch 16: 1039770.0
Loss after epoch 17: 1033054.0
Loss after epoch 18: 1027540.0
Loss after epoch 19: 1025520.0


(3873945, 6687940)

### 4 - Ensayar

##### VEMOS PALABRAS QUE SON ICÓNICAS EN EL LENGUAJE DEL AUTOR Y TIENEN UN SIGNIFICADO MUY PARTICULAR EN SUS OBRAS

In [59]:
# Palabras que MÁS se relacionan con "Argentina":
palabra = 'argentina'
print('Palabras que mas se relacionan con ', palabra)
print('\n')
w2v_model.wv.most_similar(positive=[palabra], topn=10)


Palabras que mas se relacionan con  argentina




[('liminar', 0.7941634058952332),
 ('cíclica', 0.7803575992584229),
 ('croce', 0.7772975564002991),
 ('deliberada', 0.7752829194068909),
 ('sofismas', 0.7751613855361938),
 ('contemporánea', 0.7704449892044067),
 ('previa', 0.768276572227478),
 ('platónica', 0.7677533030509949),
 ('piadosa', 0.7632854580879211),
 ('gauchesca', 0.7615621089935303)]

In [60]:
# Palabras que MENOS se relacionan con "Argentina":
print('Palabras que menos se relacionan con ', palabra)
print('\n')
w2v_model.wv.most_similar(negative=[palabra], topn=10)

Palabras que menos se relacionan con  argentina




[('monedas', -0.051038410514593124),
 ('unas', -0.05111860856413841),
 ('cuyas', -0.0651317909359932),
 ('ni', -0.09138272702693939),
 ('al', -0.09296976774930954),
 ('tus', -0.09434295445680618),
 ('hacia', -0.09645665436983109),
 ('palabras', -0.0983056128025055),
 ('otro', -0.099836066365242),
 ('chambergo', -0.10500982403755188)]

In [85]:
# Palabras que MÁS se relacionan con "infinito":
palabra = 'infinito'
print('Palabras que mas se relacionan con ', palabra)
print('\n')
w2v_model.wv.most_similar(positive=[palabra], topn=10)

Palabras que mas se relacionan con  infinito




[('ilimitado', 0.7029339075088501),
 ('indefinido', 0.6981865167617798),
 ('elevado', 0.6784825325012207),
 ('infinitesimal', 0.663684070110321),
 ('tahúr', 0.6535478830337524),
 ('subdivisiones', 0.6531436443328857),
 ('alegórico', 0.6513125896453857),
 ('variable', 0.6450049877166748),
 ('aparte', 0.6397697925567627),
 ('intrincado', 0.6364815831184387)]

In [69]:
# Palabras que MÁS se relacionan con "dios":
palabra = 'dios'
print('Palabras que mas se relacionan con ', palabra)
print('\n')
w2v_model.wv.most_similar(positive=[palabra], topn=10)

Palabras que mas se relacionan con  dios




[('demonio', 0.5407249331474304),
 ('buddha', 0.5332194566726685),
 ('tribunal', 0.5127353668212891),
 ('subalterno', 0.5101310014724731),
 ('trueno', 0.5034234523773193),
 ('arte', 0.5018683075904846),
 ('misericordioso', 0.4992934763431549),
 ('thor', 0.4987759590148926),
 ('edificio', 0.497972309589386),
 ('artificio', 0.490837037563324)]

In [70]:
# Palabras que MÁS se relacionan con "sueño":
palabra = 'sueño'
print('Palabras que mas se relacionan con ', palabra)
print('\n')
w2v_model.wv.most_similar(positive=[palabra], topn=10)

Palabras que mas se relacionan con  sueño




[('simétrico', 0.6364108324050903),
 ('vigilia', 0.6347007155418396),
 ('fluencia', 0.6320925354957581),
 ('inquieta', 0.6304628849029541),
 ('frío', 0.6303454637527466),
 ('irreversible', 0.6264906525611877),
 ('encrucijada', 0.6224621534347534),
 ('domingo', 0.6207628846168518),
 ('águila', 0.6154237985610962),
 ('tapiz', 0.6123214960098267)]

In [64]:
# Palabras que MÁS se relacionan con "muerte":
palabra = 'muerte'
print('Palabras que mas se relacionan con ', palabra)
print('\n')
w2v_model.wv.most_similar(positive=[palabra], topn=10)

Palabras que mas se relacionan con  muerte




[('vida', 0.6465276479721069),
 ('vuelta', 0.6154322028160095),
 ('prisión', 0.5882643461227417),
 ('visita', 0.5872514247894287),
 ('perra', 0.5796576738357544),
 ('garganta', 0.5761924386024475),
 ('vigilia', 0.5757505893707275),
 ('debido', 0.5747835636138916),
 ('chacarita', 0.5747679471969604),
 ('hija', 0.5720855593681335)]

In [66]:
# Palabras que MÁS se relacionan con "laberinto":
palabra = 'laberinto'
print('Palabras que mas se relacionan con ', palabra)
print('\n')
w2v_model.wv.most_similar(positive=[palabra], topn=10)

Palabras que mas se relacionan con  laberinto




[('yelmo', 0.7099664807319641),
 ('simétrico', 0.7074602842330933),
 ('metal', 0.7067455053329468),
 ('vertiginoso', 0.7047392129898071),
 ('balcón', 0.7029560804367065),
 ('cristal', 0.6934791803359985),
 ('pesado', 0.691338062286377),
 ('gato', 0.6879094243049622),
 ('sordo', 0.6868997812271118),
 ('oscuro', 0.686290979385376)]

In [74]:
# Palabras que MÁS se relacionan con "aleph":
palabra = 'aleph'
print('Palabras que mas se relacionan con ', palabra)
print('\n')
w2v_model.wv.most_similar(positive=[palabra], topn=10)

Palabras que mas se relacionan con  aleph




[('espectador', 0.7747349739074707),
 ('sótano', 0.7555372714996338),
 ('barco', 0.7526008486747742),
 ('maldonado', 0.7454703450202942),
 ('verlaine', 0.7396864295005798),
 ('cenizas', 0.7283073663711548),
 ('hacedor', 0.7244381308555603),
 ('ascetismo', 0.7203271985054016),
 ('antebrazo', 0.7164846658706665),
 ('agotado', 0.7154183387756348)]

In [72]:
# Palabras que MÁS se relacionan con "amor":
palabra = 'amor'
print('Palabras que mas se relacionan con ', palabra)
print('\n')
w2v_model.wv.most_similar(positive=[palabra], topn=10)

Palabras que mas se relacionan con  amor




[('tedio', 0.6962491273880005),
 ('pálido', 0.6852408647537231),
 ('crueldad', 0.6846229434013367),
 ('atravesar', 0.6825698614120483),
 ('purifica', 0.681066632270813),
 ('temor', 0.679689347743988),
 ('ave', 0.6760972142219543),
 ('escudo', 0.6735124588012695),
 ('tumulto', 0.6728698015213013),
 ('molino', 0.6723179817199707)]

### 5 - Visualizar agrupación de vectores

In [83]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 3  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    z_vals = [v[2] for v in vectors]

    return x_vals, y_vals, z_vals, labels

In [84]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals,z_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter_3d(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS],z=z_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.

